In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("PySparkShell") \
    .getOrCreate()

In [3]:
import pyspark.sql.functions as fn
from pyspark.sql.types import *
import string
from pyspark.sql.functions import lit
import pyspark.sql.column
import string
from pyspark.ml import Pipeline
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
import math

In [4]:
import numpy as np

# Input

In [5]:

inputBooks = sc.wholeTextFiles("project3Data/*.txt")
inputBaseFillter=inputBooks.map(lambda (BookName,Text):\
(BookName.replace("file:/home/roje/Desktop/project3/project3Data/","").replace(".txt",""),\
 str(Text.encode("utf-8-sig"))))
NumberOfFiles=inputBooks.count()

# Filters

In [6]:
import random

In [7]:
FirstJob = inputBaseFillter.map(lambda (N,T): (N, (','.join([''.join(e for e in y if e in string.printable).strip('\"') for y in T.split(',')]))\
                .replace("?","")\
                .replace("\""," ")\
                .replace("*"," ")\
                .replace("-"," ")\
                .replace("."," ")\
                .replace("!"," ")\
                .replace(","," ")\
                .replace("'"," ")\
                .replace("/"," ")\
                .replace("="," ")\
                .replace("("," ")\
                .replace(")"," ")\
                .replace(";"," ")\
                .replace("["," ")\
                .replace(":"," ")\
                .replace("'"," ")\
                .replace("]"," ")\
                .replace("+"," ")\
                .replace("_"," ")))


In [8]:
SecoundJob=FirstJob.map(lambda (N,T):(N,T.lower().replace(" ourselves "," ").replace(" hers "," ").replace(" between "," ").replace(" yourself "," ")\
.replace(" but "," ").replace(" again "," ").replace(" there "," ").replace(" about "," ")\
.replace(" once "," ").replace(" during "," ").replace(" out "," ").replace(" very "," ")\
.replace(" having "," ").replace(" with "," ").replace(" they "," ").replace(" own "," ")\
.replace(" an "," ").replace(" be "," ").replace(" some "," ").replace(" for "," ").replace(" do "," ")\
.replace(" its "," ").replace(" yours "," ").replace(" such "," ").replace(" into "," ").replace(" of "," ")\
.replace(" most "," ").replace(" itself "," ").replace(" other "," ").replace(" off "," ").replace(" is "," ")\
.replace(" s "," ").replace(" am "," ").replace(" or "," ").replace(" who "," ").replace(" as "," ")\
.replace(" from "," ").replace(" him "," ").replace(" each "," ").replace(" the "," ").replace(" themselves "," ")\
.replace(" until "," ").replace(" below "," ").replace(" are "," ").replace(" we "," ").replace(" these "," ")\
.replace(" your "," ").replace(" his "," ").replace(" through "," ").replace(" don "," ").replace(" nor "," ").replace(" me "," ").replace(" were "," ").replace(" her "," ").replace(" more "," ").replace(" himself "," ").replace(" this "," ").replace(" down "," ").replace(" should "," ").replace(" our "," ").replace(" their "," ").replace(" while "," ").replace(" above "," ").replace(" both "," ").replace(" up "," ").replace(" to "," ").replace(" ours "," ").replace(" had "," ").replace(" she "," ").replace(" all "," ").replace(" no "," ").replace(" when "," ").replace(" at "," ").replace(" any "," ").replace(" before "," ").replace(" them "," ").replace(" same "," ").replace(" and "," ").replace(" been "," ").replace(" have "," ").replace(" in "," ").replace(" will "," ").replace(" on "," ").replace(" does "," ").replace(" yourselves "," ").replace(" then "," ").replace(" that "," ").replace(" because "," ").replace(" what "," ").replace(" over "," ").replace(" why "," ").replace(" so "," ").replace(" can "," ").replace(" did "," ").replace(" not "," ").replace(" now "," ").replace(" under "," ").replace(" he "," ").replace(" you "," ").replace(" herself "," ").replace(" has "," ").replace(" just "," ").replace(" where "," ").replace(" too "," ").replace(" only "," ").replace(" myself "," ").replace(" which "," ").replace(" those "," ").replace(" i "," ").replace(" after "," ").replace(" few "," ").replace(" whom "," ").replace(" t "," ").replace(" being "," ").replace(" if "," ").replace(" theirs "," ").replace(" my "," ").replace(" against "," ").replace(" a "," ").replace(" by "," ").replace(" doing "," ").replace(" it "," ").replace(" how "," ").replace(" further "," ").replace(" was "," ").replace(" here "," ")\
.replace(" than ","").replace("\r"," ").replace("\n"," ")))

In [9]:
Indexer=SecoundJob.zipWithIndex().map(lambda ((file,contents),i):(i,file,contents))
BookAndIndex=Indexer.map(lambda (i,file,contents):(i,str(file))).distinct()

In [10]:
Done=Indexer.flatMap(lambda (i,file,contents):[(i,file, word) for word in contents.split() ])

# InvertedIndex

In [11]:
outputTF = Done.map(lambda (i,file, word): (word,[int(i)]))\
      .reduceByKey(lambda a,b: a+b).filter(lambda x: len(list(set(x[1])))>1)

In [12]:
outputInvertedIndex=outputTF.map(lambda (x,y): (x,list(set(y))))


In [13]:
InvertedIndex=outputInvertedIndex.map(lambda (word,postingList):(word,len(postingList),postingList))
InvertedIndex.take(5)

#(word,df,number of documents the word appears in, the documents)

[('sevens', 3, [0, 6, 15]),
 ('chatter', 2, [8, 15]),
 ('originality', 5, [14, 2, 3, 10, 6]),
 ('unnecessarily', 3, [16, 13, 14]),
 ('politician', 3, [16, 5, 14])]

## TF "word",20,0,0,15,10,0,(sum)
## DF "word",numberOfDocs <- inverted index

In [14]:
InvertedIndexTable = spark.createDataFrame(InvertedIndex,["word","df","postingList"])

In [15]:
##Inverted Index Table

In [16]:
InvertedIndexTable.show()

+-------------+---+--------------------+
|         word| df|         postingList|
+-------------+---+--------------------+
|       sevens|  3|          [0, 6, 15]|
|      chatter|  2|             [8, 15]|
|  originality|  5|   [14, 2, 3, 10, 6]|
|unnecessarily|  3|        [16, 13, 14]|
|   politician|  3|         [16, 5, 14]|
|     bringing| 13|[0, 2, 3, 4, 5, 6...|
|         four| 17|[0, 1, 2, 3, 4, 5...|
|       wooded|  2|             [16, 2]|
|       wooden| 10|[0, 2, 3, 4, 6, 7...|
|      broiled|  2|             [6, 15]|
|   immunities|  2|             [2, 14]|
|          270|  5|     [0, 8, 2, 3, 6]|
|          272|  5|    [0, 8, 2, 10, 6]|
|          274|  6| [0, 2, 3, 6, 8, 14]|
|          276|  6| [0, 2, 3, 6, 8, 16]|
|          278|  4|        [0, 8, 2, 6]|
|      vassals|  3|          [8, 2, 10]|
|       widger|  2|             [16, 4]|
|        chins|  2|              [3, 6]|
|        china|  9|[2, 3, 5, 6, 8, 1...|
+-------------+---+--------------------+
only showing top

# TF Table

In [17]:
TFData=[]
BooksIds=["word"]
Ids=[]
TFWordRow=[]
for WordInput in outputTF.collect():
    TFWordRow=[WordInput[0]]
    for i in range(NumberOfFiles):
        count=0
        for docId in WordInput[1]:
            if docId==i:
                count=count+1
        TFWordRow.append(float(count))
    TFData.append(TFWordRow)

for i in range(NumberOfFiles):
    BooksIds.append(str(i))
    Ids.append(str(i))

In [18]:
TfTable=spark.createDataFrame(TFData,BooksIds)

In [19]:
TfTable.show()

+-------------+----+---+----+----+----+---+----+---+----+---+----+----+---+----+----+-----+----+
|         word|   0|  1|   2|   3|   4|  5|   6|  7|   8|  9|  10|  11| 12|  13|  14|   15|  16|
+-------------+----+---+----+----+----+---+----+---+----+---+----+----+---+----+----+-----+----+
|       sevens| 1.0|0.0| 0.0| 0.0| 0.0|0.0| 7.0|0.0| 0.0|0.0| 0.0| 0.0|0.0| 0.0| 0.0|  2.0| 0.0|
|      chatter| 0.0|0.0| 0.0| 0.0| 0.0|0.0| 0.0|0.0| 2.0|0.0| 0.0| 0.0|0.0| 0.0| 0.0|  1.0| 0.0|
|  originality| 0.0|0.0| 1.0| 2.0| 0.0|0.0| 5.0|0.0| 0.0|0.0| 1.0| 0.0|0.0| 0.0| 1.0|  0.0| 0.0|
|unnecessarily| 0.0|0.0| 0.0| 0.0| 0.0|0.0| 0.0|0.0| 0.0|0.0| 0.0| 0.0|0.0| 1.0| 1.0|  0.0| 3.0|
|   politician| 0.0|0.0| 0.0| 0.0| 0.0|1.0| 0.0|0.0| 0.0|0.0| 0.0| 0.0|0.0| 0.0| 1.0|  0.0| 2.0|
|     bringing| 3.0|0.0|22.0| 2.0|20.0|1.0|12.0|9.0| 3.0|0.0| 0.0| 0.0|4.0| 3.0| 2.0| 24.0|19.0|
|         four|97.0|5.0|78.0|40.0|80.0|7.0|99.0|7.0|10.0|2.0|37.0|42.0|4.0|71.0|28.0|328.0|82.0|
|       wooded| 0.0|0.0| 1.0| 

# IDF

In [20]:
IDFTableData=InvertedIndex.map(lambda (x,y,z): (x,y,z,math.log10(float(NumberOfFiles)/float(y))))

In [21]:
IDFTable=spark.createDataFrame(IDFTableData,["word","df","postingList","IDF"])

In [22]:
IDFTable[["word","IDF"]].show()

+-------------+-------------------+
|         word|                IDF|
+-------------+-------------------+
|       sevens| 0.7533276666586115|
|      chatter| 0.9294189257142927|
|  originality| 0.5314789170422551|
|unnecessarily| 0.7533276666586115|
|   politician| 0.7533276666586115|
|     bringing|0.11650556907143717|
|         four|                0.0|
|       wooded| 0.9294189257142927|
|       wooden| 0.2304489213782739|
|      broiled| 0.9294189257142927|
|   immunities| 0.9294189257142927|
|          270| 0.5314789170422551|
|          272| 0.5314789170422551|
|          274| 0.4522976709946303|
|          276| 0.4522976709946303|
|          278| 0.6283889300503115|
|      vassals| 0.7533276666586115|
|       widger| 0.9294189257142927|
|        chins| 0.9294189257142927|
|        china|0.27620641193894907|
+-------------+-------------------+
only showing top 20 rows



# TF-IDF

In [23]:
TfIDFInProgress=IDFTable.join(TfTable,"word").drop('df').drop('postingList')

In [24]:
Tester1=TfIDFInProgress.rdd

In [25]:
def RowMapForTfIdf(tup):
    RowData=[tup[0]]
    index=0
    for i in tup:
        if index>1:
            if float(tup[index])!=0:
                temp=float(float(1.0+math.log10(float(tup[index])))*float(tup.IDF))
                if temp>0:
                    RowData.append(float(float(int(temp*100))/100))
            else:
                RowData.append(float(0))
        index=index+1
    return RowData
TFIDFRDD=Tester1.map(RowMapForTfIdf)#.filter(lambda x: x[0]!="shouldBeRemovedCode1") [term]

In [26]:
TF_IDF_RDD=TFIDFRDD.filter(lambda x: len(x)>1)

In [27]:
TF_IDF=spark.createDataFrame(TF_IDF_RDD,BooksIds)

In [28]:
TF_IDF.show()

+-------------+----+---+----+----+----+----+----+----+----+---+----+----+----+----+----+----+----+
|         word|   0|  1|   2|   3|   4|   5|   6|   7|   8|  9|  10|  11|  12|  13|  14|  15|  16|
+-------------+----+---+----+----+----+----+----+----+----+---+----+----+----+----+----+----+----+
|          296|0.45|0.0|0.76|0.76|0.45| 0.0|1.12| 0.0| 0.8|0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0|
|          467| 0.0|0.0| 1.2| 0.0| 0.0| 0.0|1.81| 0.0| 0.0|0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0|
|          675| 0.0|0.0|0.92| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0|0.0| 0.0| 0.0| 0.0| 0.0|0.92| 0.0| 0.0|
| accumulation|0.62|0.0| 0.0| 0.0| 0.0| 0.0|0.62| 0.0|0.62|0.0| 0.0| 0.0| 0.0| 0.0|0.81| 0.0| 0.0|
|    ammonites|0.62|0.0| 0.0| 0.0| 0.0| 0.0|0.92| 0.0| 0.0|0.0| 0.0| 0.0|0.62| 0.0| 0.0|1.48| 0.0|
|apprehensions| 0.0|0.0|1.11| 0.0| 0.0| 0.0|0.81| 0.0| 0.0|0.0| 0.0| 0.0| 0.0| 0.0|0.62| 0.0|1.06|
|    arguments| 0.0|0.0|0.47|0.29| 0.4| 0.0|0.39| 0.0|0.34|0.0|0.39| 0.0|0.34| 0.0|0.36|0.23| 0.4|
|     ashk

# Cosine Table

In [29]:
def LogTFTable(tup):
    RowData=[str(tup[0])]
    index=0
    for i in tup:
        if index>0:
            if tup[index]!=0:
                RowData.append(1.0+math.log10(tup[index]))
            else:
                RowData.append(float(0))
        index=index+1
    return RowData
LogTFRDD=TfTable.rdd.map(LogTFTable)
LogTFRDD
TF_DataFrame=spark.createDataFrame(LogTFRDD,BooksIds)

In [30]:
temp=0
docsVec=[]
for i in BooksIds:
    if i!="word":
        vals=TF_DataFrame[[str(i)]].rdd.filter(lambda z: z[0]!=0.0).map(tuple).map(lambda t: t[0])
        for tup in vals.collect():
            temp=temp+math.pow(tup,2)
        docsVec.append(1/math.pow(temp,0.5))
        temp=0

In [31]:

CosTable=[]
for i in LogTFRDD.collect():
    row=[i[0]]
    index=1
    while index<NumberOfFiles+1:
        row.append(i[index]*docsVec[index-1])
        index=index+1
    CosTable.append(row)


In [32]:
Cos_DataFrame=spark.createDataFrame(CosTable,BooksIds)

In [33]:
temp=0
CosSimilartyWithDoc1=[]
for CheckSimilartyForBookNumber in BooksIds:
    if CheckSimilartyForBookNumber!="word":
        for i in BooksIds:
            if i!="word":
                if int(i)>int(CheckSimilartyForBookNumber):
                    if i!=str(CheckSimilartyForBookNumber):
                        vals=Cos_DataFrame[[str(CheckSimilartyForBookNumber),i]].rdd.map(tuple)
                        for tup in vals.collect():
                            temp=float(tup[0])*float(tup[1])+float(temp)
                        CosSimilartyWithDoc1.append((CheckSimilartyForBookNumber,str(i),temp))
                        temp=0

In [34]:
CosSimilartyWithDocTable=spark.createDataFrame(CosSimilartyWithDoc1,["book1","book2","rank"])

In [35]:
CosSimilartyWithDocTable.orderBy('rank', ascending=False).show()

+-----+-----+------------------+
|book1|book2|              rank|
+-----+-----+------------------+
|    2|    6|0.7458061960541694|
|    3|    6|0.7260409436314755|
|    0|    6|0.7122226075611113|
|    6|    8|0.7062522479188156|
|    2|    3|0.7019142121709496|
|    2|   14|0.6977176570070522|
|   11|   13|0.6880136221856791|
|    2|   10|0.6842634185925098|
|    4|   15|0.6773668084773767|
|    2|    8|0.6770798144221256|
|    3|    8|0.6736129823298831|
|    2|   16| 0.668850521749075|
|    7|   15| 0.660373639437973|
|    0|    3|0.6587386267026158|
|   10|   14|0.6553075101443503|
|    4|    6| 0.654443062428649|
|   14|   16|0.6533373040313876|
|    1|   11|0.6526886881132524|
|    0|    2|0.6521038396318022|
|    2|    4|0.6516589539999386|
+-----+-----+------------------+
only showing top 20 rows



In [36]:
BookAndIndex.collect()

[(7, 'The Wonder Book of Bible Stories'),
 (11, 'Girls and Athletics by Mary C. Morgan'),
 (4,
  'The Wars of the Jews; Or, The History of the Destruction of Jerusalem by Josephus'),
 (0, 'The Astronomy of the Bible by E Walter Maunder'),
 (15, 'The King James Version of the Bible'),
 (3, 'The Jew, The Gypsy and El Islam by Sir Richard Francis Burton'),
 (14,
  'Black and White Land Labor and Politics in the South by Timothy Thomas Fortune'),
 (12,
  'The History of the Ten Lost Tribes Anglo Israelism Examined by David Baron'),
 (8, 'The Jews among the Greeks and Romans by Max Radin'),
 (2,
  'History of the Negro Race in America From 1619 to 1880 Vol 1 by Williams'),
 (16, 'On War by Carl von Clausewitz'),
 (9,
  'The Black Man the Father of Civilization Proven by Biblical History by Webb'),
 (6, 'Bible Myths and their Parallels'),
 (1, 'Association Football and How To Play It'),
 (5, 'Sport The Gold Bat'),
 (10, 'The Negro by W E B Du Bois'),
 (13, 'Outdoor Sports and Games by Claude

# Query Similarty

In [37]:
#NumberOfApp*IDF*vector


def QuerySimilarty(Query):
    WordXIDFxW=[]
    WordXIDFxWxCW=[]
    DocIndex=0
    QueryRdd = sc.parallelize([("Query",Query.lower())])
    QueryWordsSplited=QueryRdd.flatMap(lambda (file,contents):[(word,1) for word in contents.split()]).reduceByKey(lambda a,b: a+b)
    #[(term1,tf),(term2,tf)]
    for tup in QueryWordsSplited.collect():
        CheckNotEmpty=GetIDF(tup[0])
        if CheckNotEmpty>0:
            if CheckNotEmpty!=0:
                WordXIDFxW.append([tup[0], float(tup[1]) *GetIDF(tup[0])])                
    #[(term1,tf*IDF),(term2,tf*IDF),(term3,tf*IDF)]
    for tup in WordXIDFxW:
        DocIndex=0
        while DocIndex<NumberOfFiles:
            WordXIDFxWxCW.append([str(DocIndex),tup[0],float(tup[1])*float(GetCosValOfWordInDoc([tup[0],DocIndex]))]) 
            DocIndex=DocIndex+1
    #[(docId1,term1,tf*IDF*Vec),(docId1,term2,tf*IDF*Vec)]
    return GetBest10(WordXIDFxWxCW)

def GetIDF(Term):
    temp=IDFTableData.filter(lambda x: x[0]==Term).collect()
    if temp!=[]:
        return IDFTableData.filter(lambda x: x[0]==Term).collect()[0][3]
    else: 0

def GetCosValOfWordInDoc(DocXTerm):
    return Cos_DataFrame.rdd.filter(lambda x: x[0]==DocXTerm[0]).collect()[0][str(DocXTerm[1])]

def GetBest10(DocXtermXscore):
    ScoreDataFrame=spark.createDataFrame(DocXtermXscore,["id","term","vector"])
    ScoreDataFrame=ScoreDataFrame.drop("term")
    RDDScoreDataFrame=ScoreDataFrame.rdd.map(tuple).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>0)
    ScoreDataFrame=spark.createDataFrame(RDDScoreDataFrame,["id","Vector"]).orderBy('vector', ascending=False)
    
    return ScoreDataFrame.show(10)
    
print("the black community suffered from racism")
QuerySimilarty("the black community suffered from racism")
print("devine punishment")
QuerySimilarty("devine punishment")
print("israel")
QuerySimilarty("israel")
print("training camp")
QuerySimilarty("training camp")
print("competition")
QuerySimilarty("competition")
print("war")
QuerySimilarty("war")
print("you play until the last minute")
QuerySimilarty("you play until the last minute")
print("you win some you lose some")
QuerySimilarty("you win some you lose some")
print("he war lingered for days on end")
QuerySimilarty("he war lingered for days on end")
print("brilliant strategy")
QuerySimilarty("brilliant strategy")

the black community suffered from racism
+---+--------------------+
| id|              Vector|
+---+--------------------+
|  8| 0.00607094571234321|
| 10|0.006030110099554556|
|  3|0.005581351642529...|
| 14|0.005342092092682165|
|  5|0.004936375099129895|
| 16|0.004758775465715...|
|  2| 0.00473344003184238|
|  6|0.004307981684093991|
|  0|0.003829271241210...|
| 12|0.003549828926600898|
+---+--------------------+
only showing top 10 rows

devine punishment
+---+--------------------+
| id|              Vector|
+---+--------------------+
| 12|0.002319904132610091|
|  4|0.002162164197196683|
|  7|0.001877536080726...|
|  8|0.001764238011328...|
|  2|0.001649728756195...|
| 14|0.001624166864780...|
| 10|0.001554216375240...|
|  3|0.001523182725923...|
| 15|0.001516102686988...|
|  1|0.001451885138153...|
+---+--------------------+
only showing top 10 rows

israel
+---+--------------------+
| id|              Vector|
+---+--------------------+
| 12|0.009254745594635657|
|  7| 0.0069478157

## 5) Subjects : Sport, History, Sience

In [38]:
#ultimate form
Tempo=Cos_DataFrame.drop("word")
SizeOfVector=Tempo[["1"]].rdd.map(tuple).count()
Doc=[]
for i in Ids:
    Docs=[i]
    DocsIds=[]
    col=Tempo[[i]].rdd.map(tuple)
    for row in col.collect():
        DocsIds.append(row[0])
    Docs.append(DocsIds)
    Doc.append(Docs)
DocsVectorsTable=spark.createDataFrame(Doc)
DocsVectors=DocsVectorsTable.rdd.map(tuple).map(lambda x: (x[0],np.array(x[1])))


#(DOCID,[vec])

# we could do this algoritem without np But It took more then 30 min to run 
we can replace every small calc on the vector with function, but np works much faster

In [39]:
def getVector (input):
    return DocsVectors.filter(lambda x: x[0]==str(input)).collect()[0][1]

In [40]:
VectorsList=[]
for Id in Ids:
    VectorsList.append(getVector(Id))
EmptyVec=[]
i=0
while i<SizeOfVector:
    EmptyVec.append(0)
    i=i+1
EmptyVec=np.array(EmptyVec)

In [41]:
def CheckSimilarty (v1,v2):
    v=v1*v2
    sumV=0
    for i in v:
        sumV=sumV+i
    return sumV

def avgVec (v):
    tv=EmptyVec
    bol=0
    for i in v:
        tv=tv+getVector(i)
    return tv/len(v)


def getEmptyCluster(inK):
    counterForEmpty=0
    Temp=[]
    while counterForEmpty<inK:
        Temp.append([])
        counterForEmpty=counterForEmpty+1
    return NewCluster

def getRandomKnumbers(input,limit):
    return random.sample(range(0, limit+1), input)

def getEmptyList ():
    return []

def calcCentroide(ListOfVectorIndexes):
    tv=EmptyVec
    bol=0
    for i in ListOfVectorIndexes:
        tv=tv+getVector(i)
    return tv/len(ListOfVectorIndexes)

In [42]:
def run1KmeanClustering(k, numOfIterations):
    arr=[]
    innerArr=[]
    Cluster=[]
    NewCluster=[]
    TempCluster=[]
    i=0
    
    RandomArr=getRandomKnumbers(k, NumberOfFiles-1)
    while i<k:
        arr.append([VectorsList[RandomArr[i]]])
        Cluster.append([str(RandomArr[i])])
        NewCluster.append([])
        TempCluster.append([])
        i=i+1
    i=0

#  I choose random files they already not in order
#Now lets beggin
    VectorId=-1
    ClusterId=-1
    while i<numOfIterations:                                    #4
        Centroids=[]
        for c in Cluster:
            Centroids.append(calcCentroide(c))
                
        for vec in VectorsList:
            VectorId=VectorId+1
            bestCluster=[0,0]
        ####################################################
            for cluster in Cluster:                             #3
                ClusterId=ClusterId+1
                for Checker in cluster:
                    Rank=CheckSimilarty(vec,Centroids[ClusterId])
                    
                if Rank>bestCluster[1]:
                    bestCluster=[]
                    bestCluster.append(ClusterId)
                    bestCluster.append(Rank)
        ####################################################
            TempCluster[bestCluster[0]].append(str(VectorId))
            ClusterId=-1
    ########################################################
        j=0
        VectorId=-1
        for x in TempCluster:
            arr[j]=[]
            for n in x:
                arr[j].append(VectorsList[int(n)-1])
            j=j+1
        i=i+1
        Cluster=TempCluster
        counterForEmpty=0
        TempCluster=[]
        while counterForEmpty<k:
            TempCluster.append([])
            counterForEmpty=counterForEmpty+1
    return Cluster
    

In [43]:
def RankCluster (ArrOfIndexes):
    rank=0
    avg=avgVec(ArrOfIndexes)
    for index in ArrOfIndexes:
        rank=rank+CheckSimilarty(getVector(index),avg)
    return rank

def RankClustering (ArrOfClusters):#k clusters
    Rank=0
    for cluster in ArrOfClusters:
        Rank=RankCluster(cluster)+Rank 
    return Rank

def KMeanAlgoritem(K_Clusters,NumOfTests):
    TestsResults=[]
    ClusteringRank=[]
    counterKMeanTests=0
    RankMax=0
    while counterKMeanTests<NumOfTests:
        TestsResults.append(run1KmeanClustering(K_Clusters, 3))#(k,numberOfClusteringFix)
        counterKMeanTests=counterKMeanTests+1
    #Now we have many clustering options
    for Test in TestsResults:
        RankIt=RankClustering(Test)
        if RankIt>RankMax:
            RankMax=RankIt
            res=Test
    return res
    

In [44]:
resu=KMeanAlgoritem(3,10)#(k,NUMBER-OF-RUNS)

In [45]:
resu

[['0', '2', '3', '4', '6', '8', '10', '14', '15', '16'],
 ['1', '5', '11', '13'],
 ['7', '9', '12']]

In [46]:
for r in resu:
    for i in r:
        print(BookAndIndex.filter(lambda (x,y): str(x)==str(i)).collect()[0][1])
    print(" ")

The Astronomy of the Bible by E Walter Maunder
History of the Negro Race in America From 1619 to 1880 Vol 1 by Williams
The Jew, The Gypsy and El Islam by Sir Richard Francis Burton
The Wars of the Jews; Or, The History of the Destruction of Jerusalem by Josephus
Bible Myths and their Parallels
The Jews among the Greeks and Romans by Max Radin
The Negro by W E B Du Bois
Black and White Land Labor and Politics in the South by Timothy Thomas Fortune
The King James Version of the Bible
On War by Carl von Clausewitz
 
Association Football and How To Play It
Sport The Gold Bat
Girls and Athletics by Mary C. Morgan
Outdoor Sports and Games by Claude Harris Miller
 
The Wonder Book of Bible Stories
The Black Man the Father of Civilization Proven by Biblical History by Webb
The History of the Ten Lost Tribes Anglo Israelism Examined by David Baron
 


In [47]:
BookAndIndex.collect()

[(7, 'The Wonder Book of Bible Stories'),
 (11, 'Girls and Athletics by Mary C. Morgan'),
 (4,
  'The Wars of the Jews; Or, The History of the Destruction of Jerusalem by Josephus'),
 (0, 'The Astronomy of the Bible by E Walter Maunder'),
 (15, 'The King James Version of the Bible'),
 (3, 'The Jew, The Gypsy and El Islam by Sir Richard Francis Burton'),
 (14,
  'Black and White Land Labor and Politics in the South by Timothy Thomas Fortune'),
 (12,
  'The History of the Ten Lost Tribes Anglo Israelism Examined by David Baron'),
 (8, 'The Jews among the Greeks and Romans by Max Radin'),
 (2,
  'History of the Negro Race in America From 1619 to 1880 Vol 1 by Williams'),
 (16, 'On War by Carl von Clausewitz'),
 (9,
  'The Black Man the Father of Civilization Proven by Biblical History by Webb'),
 (6, 'Bible Myths and their Parallels'),
 (1, 'Association Football and How To Play It'),
 (5, 'Sport The Gold Bat'),
 (10, 'The Negro by W E B Du Bois'),
 (13, 'Outdoor Sports and Games by Claude